# Static earthquake design problem to AS 5100.2-2017

## Problem statement
You are designing a two-span PSC plank bridge over a local creek to replace an existing bridge carrying a local council road in the Sutherland Shire Council area in NSW

The bridge is approximately 6 m wide, has two equal spans of 15 m and is primarily supported by 535 deep PSC plank members. There is a 150 mm thick concrete deck with a waterproof membrane. Joints are strip seals at both abutments and the pier. The bridge has no skew. Girders sit on an elastomeric bearing strip.

Abutment and pier headstocks are 1200x1000 reinforced concrete elements. The abutment and piers are supported on 2x 750 diameter piles/columns. Abutment piles are embedded 2 m into embankment fill and 2 m into rock. Pier piles are embedded 2 m into rock. Columns are 2 m tall.

## Questions
1. Determine the bridge earthquake design category
2. What is the level of analysis required?
3. What do you recommend the design performance level of the bridge be?
4. Determine the bridge frame horizontal earthquake force
5. Determine the bridge frame vertical earthquake force
6. Describe how you would use these forces to determine the earthquake design actions on bridge members.

### 1. Determine the earthquake design category
* Looking at Clause 15.4 of AS 5100.2-2017
* Adopt **BEDC-2** since our bridge has two or more spans, and is not considered to have a high volume of traffic

### 2. What is the level of analysis required?
* Consult Clause 15.4.3 of AS 5100.2-2017
* We can determine earthquake actions using either static analysis or dynamic analysis
* We'll use the **static analysis** in this instance

### 3. What do you recommend the design performance level of the bridge be?
* See Clause 15.5 of AS 5100.2-2017
* Selecting **damage control performance level** since we have decided this structure is not essential for post-disaster use

### 4. Determine the bridge frame horizontal earthquake force
#### 4.1. Fundamental period
The first thing we want to do is calculate the natural period of the structure. We could do this with a global finite element model, or using a simplified method. We'll use the simplified approach in Clause 6.2.3 of AS 1170.4-2007.

$$ T_1 = 1.25k_th_n^{0.75} $$

The $ k_t $ factors will be different for the longitudinal and lateral directions we're considering.

In [486]:
# kt factor in the longitudinal direction. A concrete frame, and not moment-resisting
kt_long = 0.05

# kt factor in the lateral direction. The headstock forms a moment-resisting frame
kt_lat = 0.075

The height of the structure, $ h_n $, is the height from the base of the structure to the uppermost seismic weight or mass, **in metres**. Here, we're adopting the "base of the structure" to be the location of fixity in the socket, which we'll say is half-way into the rock.

In [487]:
# Total length of pile in rock
D_pile = 2

# Height of pile considered for 'base'
h_pile = D_pile / 2

# Height from top of pile to top of column, in metres
h_col = 2

# Height of headstock in m
D_headstock = 1

# Thickness of a bearing strip in m
t_strip = 0.05

# Depth of PSC girder in m
D_girder = 0.530

# Thickness of deck slab in m
t_slab = 0.150

# Calculating `h_n` value. Taking the "uppermost seismic weight" as the deck level
h_n = h_pile + h_col + D_headstock + t_strip + D_girder + t_slab

Now the fundamental period of the structure, $T_1$, for each direction considered.

In [488]:
def T1(kt, hn):
    
    val = 1.25 * kt * (hn ** 0.75)
    
    return val

T1_long = T1(kt_long, h_n)
T1_lat = T1(kt_lat, h_n)

print("Fundamental period of longitudinal direction is", round(T1_long, 2),"seconds.")
print("Fundamental period of lateral direction is", round(T1_lat, 2),"seconds.")

Fundamental period of longitudinal direction is 0.2 seconds.
Fundamental period of lateral direction is 0.3 seconds.


#### 4.2. Site sub-soil class
We now need to decide on the site sub-soil class. Generally, Section 4 of AS 1170.4-2007 provides commentary on which site sub-soil class is most appropriate.

Clause 15.7 of AS 5100.2-2017 also advises that for bridges with pile-supported foundations (like ours), the site subsoil class shall be based on the **upper layers of the soil profile**. 

This means that we may decide to adopt a subsoil class based on the embankment fill. In our case, we have fill at the abutments, but the pier is only founded in rock. Here we're going to ignore the presence of the embankment fill and adopt a site sub-soil class of $B_e$ for the foundation in rock.

#### 4.3. Spectral shape factor
With the site sub-soil class and fundamental period, we can now determine the spectral shape factor using either Table 6.4 or Figure 6.4 in AS 1170.4-2007.

While we could write a function to more generally determine this value, in our case, the fundamental periods are sufficiently low to read the values off the table directly.

In [489]:
ChT_long = 2.94
ChT_lat = 2.94

#### 4.4. Annual probability of exceedance and probability factor
The probability factor, $k_p$ is determined in accordance with Clause 15.6 of AS 5100.2-2017. It is based on the **annual probability of exceedance** in Table 15.6. The engineer then refers to Table 3.1 of AS 1170.4-2007 to determine the **probability factor**.

In our case, the annual probability of exceedance, $P$, is $\frac{1}{500}$ (Table 15.6).

In [490]:
# Probability factor in accordance with Table 3.1 of AS 1170.4-2007
kp = 1.0

#### 4.5. Hazard design factor
Similarly, the hazard design factor is determined in accordance with either Table 3.2 or Figures 3.2(A) through 3.2(F) of AS 1170.4-2007. Given our location is in the Sutherland Shire of NSW:

In [491]:
# Hazard design factor
Z = 0.09

#### 4.6. Seismic acceleration for _elastic_ horizontal earthquake response
Calculating the design action coefficient in accordance with Clause 15.9.1 of AS 5100.2-2017.

$$ C(T_f) = k_pZC_h(T_f) $$

Where:
* $k_p$ = probability factor, calculated above
* $Z$ = design seismic hazard factor, calculated above
* $C_h(T_f)$ = acceleration spectral shape factor at fundamental natural period of vibration of the bridge frame in the direction considered (longitudinal or transverse)

In [492]:
def CTf(kp, Z, ChTf):
    
    val = kp * Z * ChTf
    
    return val

CTf_long = CTf(kp, Z, ChT_long)
CTf_lat = CTf(kp, Z, ChT_lat)

print("Coefficient for elastic horizontal response in the longitudinal direction is", round(CTf_long, 3))
print("Coefficient for elastic horizontal response in the lateral direction is", round(CTf_lat, 3))

Coefficient for elastic horizontal response in the longitudinal direction is 0.265
Coefficient for elastic horizontal response in the lateral direction is 0.265


#### 4.7. Design ductility factor
The design ductility factor, $\mu$, will differ for the two directions considered (longitudinal and lateral). To determine these values, we consult Table 15.9.2 of AS 5100.2-2017 and its notes.

From Question 3 above, we have decided the performance level is **damage control**, so we'd select the appropriate column in the table.

For the longitudinal direction, the girders sit on an elastomeric bearing strip providing no translational restraint, so it is most appropriate for us to adopt the configuration:
> Superstructure on elastomeric bearings without translational movement restraint in the direction considered

For the lateral direction, it is not as simple. In the problem statement, we have not stated there is any lateral restraint (whereas in reality there definitely should be). If this is the case, we'd adopt the same ductility factor as the longitudinal direction.

However, if we assume there is a lateral restraint, then you might find it falls into one of the two categories:
> Superstructures on fixed pot or spherical bearings or **elastomeric bearings** with translational movement restraint in the direction considered at piers on
* stiff foundations (i.e. with negligible translational and rotational movements)
* flexible foundations with significant contribution to the displacement at pier top (i.e. piles in 10 m or more of soft soil)

> Concrete pier/piles without stiff pile caps

We could reasonably fit into any of those categories for the lateral direction. Fortunately, Note 1 to the table tells us that in a situation where more than one category applies, we adopt the lowest design ductility factor.

The resulting factors:

In [493]:
mu_long = 1.5
mu_lat = 2.0

#### 4.8. Seismic acceleration for _ductile_ horizontal earthquake response
Calculating the design action coefficient in accordance with Clause 15.9.2 of AS 5100.2-2017.

$$ C_d(T_f) = \frac{C(T_f)}{\mu} $$

Where:
* $C(T_f)$ = design action coefficient for **elastic** horizontal earthquake response, as above
* $\mu$ = design ductility factor

In [494]:
def CdTf(CTf, mu):
    
    val = CTf / mu
    
    return val

CdTf_long = CdTf(CTf_long, mu_long)
CdTf_lat = CdTf(CTf_lat, mu_lat)

print("Coefficient for ductile horizontal response in the longitudinal direction is", round(CdTf_long, 3))
print("Coefficient for ductile horizontal response in the lateral direction is", round(CdTf_lat, 3))

Coefficient for ductile horizontal response in the longitudinal direction is 0.176
Coefficient for ductile horizontal response in the lateral direction is 0.132


#### 4.9. Bridge frame horizontal earthquake force
For each bridge frame (see Section 15.2 of AS 5100.2-2017) we will calculate the horizontal earthquake force. We would then like to apply these horizontal forces at the centroid of each seismic mass, so we'll calculate them independently and then sum them at the end for our total horizontal force.

The elements we'll consider:
* PSC girders
* Reinforced concrete deck
* Reinforced concrete headstocks
* Columns (including those in embankment fill) above the rock level

Since we need to consider each frame separately, and our bridge is symmetrical, we'll just look at one frame. Properties for the PSC girder are taken from the [SCI website](http://www.sciaust.com.au/desroadplanks.html).

$$ F_F = C_d(T_f)\sum_{i-1}^{n}W_i $$

Where:
* $W_i$ = one of the $n$ seismic weights in the bridge frame
* $C_d(T_f)$ = value of the design action coefficient for ductile response

In [495]:
import numpy as np

# Mass of a girder in kg/m
mg = 600

# Length of a girder in mm
L = 15000

# Depth of girder in mm
Dg = 535

# Width of a girder
bg = 600

# Total deck width in mm
B = 6000

# Slab thickness in mm
ts = 150

# Number of girders
ng = B / bg

# Unit weight of in-situ reinforced concrete in kN/m**3
gamma_c = 25

# Total weight of the deck, in kN
W_deck = ((gamma_c * ts * B / (1000**2)) + (mg * 9.81 / 1000 * ng)) * L / 1000

# Total weight of the headstock
D_headstock = 1000
B_headstock = 1200
W_headstock = B_headstock * D_headstock * B * gamma_c / (1000**2) / 1000

# Weight of pier columns
D_col = 750
n_col = 2
L_col = 2000
A_col = np.pi * (D_col**2) / 4 * n_col
W_cols = A_col * L_col * gamma_c / (1000**2) / 1000

# Bridge frame horizontal earthquake force in kN, in line with Clause 15.10.1 of AS 5100.2-2017
Ff_deck_long = CdTf_long * W_deck
Ff_headstock_long = CdTf_long * W_headstock
Ff_col_long = CdTf_long * W_cols
Ff_long = Ff_deck_long + (2 * Ff_headstock_long) + (2 * Ff_col_long)

Ff_deck_lat = CdTf_lat * W_deck
Ff_headstock_lat = CdTf_lat * W_headstock
Ff_col_lat = CdTf_lat * W_cols
Ff_lat = Ff_deck_lat + (2 * Ff_headstock_lat) + (2 * Ff_col_lat)

print("Bridge frame horizontal earthquake force in the longitudinal direction is", round(Ff_long,1), "kN")
print("Bridge frame horizontal earthquake force in the lateral direction is", round(Ff_lat,1), "kN")

Bridge frame horizontal earthquake force in the longitudinal direction is 294.4 kN
Bridge frame horizontal earthquake force in the lateral direction is 220.8 kN


### 5. Determine the bridge frame vertical earthquake force
* Not required for BEDC-2
* See Clause 15.4.3 of AS 5100.2-2017
* If it was required by the asset owner (not required by AS 5100) then you would take two-thirds of the design action coefficient for the **elastic** horizontal response. See Clause 15.9.3 of AS 5100.2-2017

### 6. Describe how you would use these forces to determine the earthquake design actions on bridge members.
* Model each frame separately
* Calculate the horizontal force for each seismic mass and apply at its centroid
* See Clause 15.2 of AS 5100.2-2017 for further commentary